In [1]:
from modelDSUNET import *

2025-03-16 14:29:45.213836: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-16 14:29:45.297260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742115585.331558   16712 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742115585.341419   16712 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-16 14:29:45.415961: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

shape of rgb stream=>(None, 128, 128, 64)
shape of rgb stream=>(None, 64, 64, 64)
shape of rgb stream=>(None, 32, 32, 128)
shape of rgb stream=>(None, 16, 16, 256)
shape of rgb stream=>(None, 8, 8, 512)
shape of noise stream=>(None, 128, 128, 64)
shape of noise stream=>(None, 64, 64, 64)
shape of noise stream=>(None, 32, 32, 128)
shape of noise stream=>(None, 16, 16, 256)
shape of noise stream=>(None, 8, 8, 512)
Fusion at layer 0: (None, 128, 128, 64)
Fusion at layer 1: (None, 64, 64, 64)
Fusion at layer 2: (None, 32, 32, 128)
Fusion at layer 3: (None, 16, 16, 256)
Fusion at layer 4: (None, 8, 8, 512)
shape of fused=>(None, 128, 128, 64)
shape of fused=>(None, 64, 64, 64)
shape of fused=>(None, 32, 32, 128)
shape of fused=>(None, 16, 16, 256)
shape of fused=>(None, 8, 8, 512)
(None, 16, 16, 256)
(None, 32, 32, 512)
(None, 64, 64, 256)
(None, 128, 128, 64)
(None, 256, 256, 32)
final output of the model=>(None, 256, 256, 1)


Model: "DS_UNet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ rgb_input           │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ srm_filter_layer    │ (None, 256, 256,  │          0 │ rgb_input[0][0]   │
│ (SRMFilterLayer)    │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rgb_encoder         │ [(None, 128, 128, │  4,847,168 │ rgb_input[0][0]   │
│ (Functional)        │ 64), (None, 64,   │            │                   │
│                     │ 64, 64), (None,   │            │                   │
│                     │ 32, 32, 128),     │            │                   │
│                     │ (None, 16, 16,    │            │                   │
│                     │ 256), (None, 8,   │            │                   │
│                     │ 8, 512)]          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ noise_encoder       │ [(None, 128, 128, │  4,847,168 │ srm_filter_layer… │
│ (Functional)        │ 64), (None, 64,   │            │                   │
│                     │ 64, 64), (None,   │            │                   │
│                     │ 32, 32, 128),     │            │                   │
│                     │ (None, 16, 16,    │            │                   │
│                     │ 256), (None, 8,   │            │                   │
│                     │ 8, 512)]          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fusion4 (Add)       │ (None, 8, 8, 512) │          0 │ rgb_encoder[0][4… │
│                     │                   │            │ noise_encoder[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 16, 16,    │    524,544 │ fusion4[0][0]     │
│ (Conv2DTranspose)   │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fusion3 (Add)       │ (None, 16, 16,    │          0 │ rgb_encoder[0][3… │
│                     │ 256)              │            │ noise_encoder[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 16, 16,    │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 512)              │            │ fusion3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 16, 16,    │  1,179,904 │ concatenate[0][0] │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ in_place_abn        │ (None, 16, 16,    │      1,024 │ conv2d_1[0][0]    │
│ (InPlaceABN)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 16, 16,    │    590,080 │ in_place_abn[0][… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ in_place_abn_1      │ (None, 16, 16,    │      1,024 │ conv2d_2[0][0]    │
│ (InPlaceABN)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 16, 16,    │    590,080 │ in_place_abn_1[0… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 18,631,629 (71.07 MB)

 Trainable params: 13,777,741 (52.56 MB)

 Non-trainable params: 4,853,888 (18.52 MB)

done hai 


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import random

import os

Physical devices cannot be modified after being initialized


In [3]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')





In [4]:
# Set Paths
train_path = r"/home/manik/Documents/datasets/casia_dataset/train"
val_path = r"/home/manik/Documents/datasets/casia_dataset/val"

# Set Parameters
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 16

In [5]:
def get_image_mask_dataframe(image_dir, mask_dir):
    image_paths = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('png', 'jpg', 'jpeg', 'bmp', 'tiff'))])
    mask_paths = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir) if f.endswith(('png', 'jpg', 'jpeg', 'bmp', 'tiff'))])
    
    return pd.DataFrame({'image': image_paths, 'mask': mask_paths})


train_df = get_image_mask_dataframe(os.path.join(train_path, "image"), os.path.join(train_path, "masks"))
val_df = get_image_mask_dataframe(os.path.join(val_path, "image"), os.path.join(val_path, "masks"))

train_datagen = ImageDataGenerator( 
    rescale=1.0 / 255.0,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest"
)

val_test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

def create_generator(datagen, df, x_col, color_mode="rgb"):
    return datagen.flow_from_dataframe(
        dataframe=df,
        x_col=x_col,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode=None,  
        color_mode=color_mode,
        seed=42,
        shuffle=True,
        
    )



In [6]:
# Create Separate Generators for Images & Masks
train_img_gen = create_generator(train_datagen, train_df, x_col="image", color_mode="rgb")
train_mask_gen = create_generator(train_datagen, train_df, x_col="mask", color_mode="grayscale")

val_img_gen = create_generator(val_test_datagen, val_df, x_col="image", color_mode="rgb")
val_mask_gen = create_generator(val_test_datagen, val_df, x_col="mask", color_mode="grayscale")

def image_mask_generator(image_gen, mask_gen):
    while True:
        img_batch = next(image_gen)  # Get the next batch of images
        mask_batch = next(mask_gen)  # Get the next batch of masks
        yield img_batch, mask_batch 
# Zip Image and Mask Generators
train_ds = image_mask_generator(train_img_gen, train_mask_gen)
val_ds = image_mask_generator(val_img_gen, val_mask_gen)


Found 1000000 validated image filenames.
Found 1000000 validated image filenames.
Found 5000 validated image filenames.
Found 5000 validated image filenames.


In [5]:

# def display_random_sample(dataset, num_samples=1):
#     """Randomly selects and displays an image and its corresponding mask."""
#     batch_images, batch_masks = next(dataset)  
#     for _ in range(num_samples):
#         idx = random.randint(0, len(batch_images) - 1) 

#         fig, ax = plt.subplots(1, 2, figsize=(10, 5))

#         ax[0].imshow(batch_images[idx])  # Display Image
#         ax[0].set_title("Image")
#         ax[0].axis("off")

#         ax[1].imshow(batch_masks[idx].squeeze(), cmap="gray")  # Display Mask
#         ax[1].set_title("Mask")
#         ax[1].axis("off")

#         plt.show()

# # Example Usage
# display_random_sample(val_ds)


In [6]:
# batch_images, batch_masks = next(val_ds)
# batch_images.shape


In [6]:
from tensorflow.keras.optimizers import Adam
from focal_loss import BinaryFocalLoss
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

In [6]:
from metrics import *

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard

callbacks = [
        ModelCheckpoint(r"/home/manik/Documents/research/models/testing.keras", verbose=1, save_best_only=True), #when save_best_only=1,it will save model at epochs where val loss is improving,else will save at every epoch
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(r"/home/manik/Documents/research/models"),
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ]


In [ ]:
DSUNET_model.compile(optimizer="adam", loss=, 
              metrics=[F1Score(), Accuracy(), AUC(), MCC()])


In [ ]:
# train_ds = create_dual_input_generator(train_df, "image", "mask", (256,256), BATCH_SIZE)
# val_ds = create_dual_input_generator(val_df, "image", "mask", (256,256), BATCH_SIZE)

history = DSUNET_model.fit(
    train_ds,
    validation_data=val_ds,
    steps_per_epoch=len(train_df) // BATCH_SIZE,
    validation_steps=len(val_df) // BATCH_SIZE,
    epochs=10,
    callbacks=[early_stopping]
)


In [ ]:
DSUNET_model.save("best_model.keras")

In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Input, Dropout, BatchNormalization
from tensorflow.keras.models import Model

def unet(input_size=(256, 256, 3)):
    inputs = Input(input_size)

    # Encoder (Downsampling)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = BatchNormalization()(c1)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    c1 = BatchNormalization()(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    c2 = BatchNormalization()(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = BatchNormalization()(c3)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    c3 = BatchNormalization()(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = BatchNormalization()(c4)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    c4 = BatchNormalization()(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = BatchNormalization()(c5)
    c5 = Dropout(0.5)(c5)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)
    c5 = BatchNormalization()(c5)

    # Decoder (Upsampling)
    u6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = BatchNormalization()(c6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)
    c6 = BatchNormalization()(c6)

    u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = BatchNormalization()(c7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)
    c7 = BatchNormalization()(c7)

    u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = BatchNormalization()(c8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)
    c8 = BatchNormalization()(c8)

    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = BatchNormalization()(c9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)
    c9 = BatchNormalization()(c9)

    # Output layer (Sigmoid for binary segmentation)
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs, outputs)
    return model

# Create model
model = unet()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary
model.summary()


I0000 00:00:1739726018.805790    5089 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4272 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256, 256,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │     36,928 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │    147,584 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │      1,024 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 64, 64,    │    590,080 │ batch_normalizat… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │      1,024 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 32, 32,    │  1,180,160 │ max_pooling2d_2[

 Total params: 31,055,297 (118.47 MB)

 Trainable params: 31,043,521 (118.42 MB)

 Non-trainable params: 11,776 (46.00 KB)

In [11]:
# train_ds = create_dual_input_generator(train_df, "image", "mask", (256,256), BATCH_SIZE)
# val_ds = create_dual_input_generator(val_df, "image", "mask", (256,256), BATCH_SIZE)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    steps_per_epoch=len(train_df) // BATCH_SIZE,
    validation_steps=len(val_df) // BATCH_SIZE,
    epochs=10,
    callbacks=[early_stopping]
)


Epoch 1/10


E0000 00:00:1739726300.706251    5643 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
E0000 00:00:1739726300.842183    5643 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2025-02-16 22:48:20.859407: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at xla_ops.cc:577 : FAILED_PRECONDITION: DNN library initialization failed. Look at the er

FailedPreconditionError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/manik/tf-gpu/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/manik/tf-gpu/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/manik/tf-gpu/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/manik/tf-gpu/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/manik/tf-gpu/lib/python3.10/site-packages/ipykernel/kernelbase.py", line